In [ ]:
from cronjob import get_repositories
from crowdin import crowdin_download_missing_translations, crowdin_request, delete_code_translations, delete_translation, get_orphaned_files_as_data_frame
from crowdin_sync import get_repository_state, update_repository
from datetime import datetime
from file_manager import get_crowdin_file, get_local_file
import git
import json
import os
import pandas as pd
import re
from repository import initial_dir
from zendesk import download_zendesk_articles, update_zendesk_articles, update_zendesk_translation, zendesk_get_request

In [ ]:
repositories_df = pd.read_csv('repositories.csv', comment='#')
repositories_df

In [ ]:
def get_update_result(index):
    repository = get_repositories(False)[index]
    return repository, get_repository_state(repository)

# Update liferay-docs

In [ ]:
repository, update_result = get_update_result(4)

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
crowdin_download_missing_translations(repository, all_files, file_info)

In [ ]:
uat_domain = 'liferaysupport1528999723.zendesk.com'
prod_domain = 'liferay-support.zendesk.com'

In [ ]:
from zendesk_docs import get_translation_target_articles

In [ ]:
article_ids, articles = get_translation_target_articles(prod_domain, repository, file_info)

In [ ]:
for key, value in article_ids.items():
    if value is None:
        print(key)

# Test updating ignore list

In [ ]:
def add_ignore_files(repository, folder):
    os.chdir(repository.github.git_root)
    translated_files = get_eligible_files(repository, git.ls_files(folder), 'ja')
    os.chdir(initial_dir)

    source_files = [
        'en/' + file[3:] if file[0:3] == 'ja/' else file.replace('/ja/', '/en/')
            for file in translated_files
    ]
    
    entries = set()

    for x in pd.read_csv('%s/ignore.csv' % initial_dir).to_records(index=False):
        entries.add('%s/%s' % (x[0], x[1]))

        if x[1][-3:] == '.md':
            entries.add('%s/%s.markdown' % (x[0], x[1][:-3]))
    
    for x in source_files:
        entries.add('%s/%s' % (repository.github.upstream, x))

        if x[-3:] == '.md':
            entries.add('%s/%s.markdown' % (repository.github.upstream, x[:-3]))

    def get_ignore_entry(path):
        remote, repo_name, file = path.split('/', 2)
        return { 'repository': remote + '/' + repo_name, 'file': file }
    
    df = pd.DataFrame(sorted(
        [get_ignore_entry(x) for x in entries],
        key = lambda x: (x['repository'], x['file']
    )))
    
    df.to_csv('%s/ignore.csv' % initial_dir, index=False)

In [ ]:
repository, update_result = get_update_result(4)
assert(repository.github.origin == 'holatuwol/liferay-docs')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
add_ignore_files(repository, 'ja/develop/tutorials/articles/01-introduction-to-liferay-development')
add_ignore_files(repository, 'ja/develop/tutorials/articles/100-tooling')
add_ignore_files(repository, 'ja/develop/tutorials/articles/120-customizing')
add_ignore_files(repository, 'ja/develop/tutorials/articles/140-application-security')
add_ignore_files(repository, 'ja/develop/tutorials/articles/150-web-services')
add_ignore_files(repository, 'ja/develop/tutorials/articles/160-search')
add_ignore_files(repository, 'ja/develop/tutorials/articles/185-liferay-js-apis')
add_ignore_files(repository, 'ja/develop/tutorials/articles/210-front-end-taglibs')
add_ignore_files(repository, 'ja/develop/tutorials/articles/240-product-navigation')
add_ignore_files(repository, 'ja/develop/tutorials/articles/280-workflow')
add_ignore_files(repository, 'ja/develop/tutorials/articles/300-internationalization')
add_ignore_files(repository, 'ja/develop/tutorials/articles/310-application-display-templates')
add_ignore_files(repository, 'ja/develop/tutorials/articles/380-troubleshooting')

# Test RST file updates

In [ ]:
repository, update_result = get_update_result(6)
assert(repository.github.origin == 'holatuwol/liferay-learn')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
from crowdin_sync import update_sources

os.chdir(repository.github.git_root)
rst_files = [file for file in all_files if file[-4:] == '.rst']
update_sources(repository, rst_files)
os.chdir(initial_dir)

In [ ]:
os.chdir(repository.github.git_root)
for file in file_info.keys():
    if file.find(repository.crowdin.dest_folder) == 0 and file[-4:] == '.rst':
        local_file = get_local_file(repository, file)
        delete_code_translations(repository, local_file, file_info)
os.chdir(initial_dir)

# Test changes to updates

In [ ]:
repository = get_repositories(False)[6]
assert(repository.github.origin == 'holatuwol/liferay-learn')

In [ ]:
update_result = get_repository_state(repository)

In [ ]:
update_result = update_repository(repository)

In [ ]:
uat_domain = 'liferaysupport1528999723.zendesk.com'
prod_domain = 'liferay-support.zendesk.com'

In [ ]:
audits = zendesk_get_request(prod_domain, '/tickets/22226/audits.json', 'audit')

# Update ZenDesk articles

In [ ]:
uat_domain = 'liferaysupport1528999723.zendesk.com'
prod_domain = 'liferay-support.zendesk.com'

In [ ]:
repository = get_repositories(False)[0]
assert(repository.github.origin == 'holatuwol/zendesk-articles')

In [ ]:
assert(repository.github.single_folder == 'en/Fast Track')

In [ ]:
# import urllib

# zendesk_articles = download_zendesk_articles(repository, prod_domain)

# count = 0

# for article in zendesk_articles.values():
#     if article['locale'] == 'ja':
#         print(article['label_names'])
#         count = count + 1
#         print(urllib.parse.unquote(article['html_url']))

# print(count)

In [ ]:
zendesk_articles = update_zendesk_articles(repository, uat_domain)

In [ ]:
df1 = pd.DataFrame([
    {'user_segment_id': str(segment['id']), 'user_type': segment['user_type']}
        for segment in zendesk_get_request(uat_domain, '/help_center/user_segments.json', 'user_segments')
])

In [ ]:
df1

In [ ]:
df2 = pd.DataFrame([
    {'article_id': str(article['id']), 'user_segment_id': str(article['user_segment_id'])}
         for article in zendesk_articles.values()
])

df3 = df2.groupby('user_segment_id').count()

In [ ]:
df3

In [ ]:
df1.join(df3, on='user_segment_id', how='right')

# Delete repository from CrowdIn

In [ ]:
repository, update_result = get_update_result(0)
assert(repository.github.origin == 'holatuwol/zendesk-articles')

In [ ]:
prefix = repository.crowdin.dest_folder + '/'

for key, value in file_info.items():
    if 'id' not in value and key.find(prefix) == 0 and key.find('/', len(prefix)) != -1:
        delete_translation(repository, key)

# Remove code blocks

In [ ]:
repository, update_result = get_update_result(1)
assert(repository.github.origin == 'holatuwol/liferay-docs')

In [ ]:
new_files, all_files, file_info = update_result

In [ ]:
for file in all_files:
    delete_code_translations(repository, file, file_info)

# Check orphaned files statistics

In [ ]:
repository, update_result = get_update_result(3)
assert(repository.github.origin == 'holatuwol/liferay-learn')

In [ ]:
df = get_orphaned_files_as_data_frame(repository, update_result)

In [ ]:
def get_crowdin_link(file_id):
    return '<a href="https://crowdin.com/translate/liferay-documentation/%s/en-ja">%s</a>' % (file_id, file_id)

In [ ]:
df.style.format({'id1': get_crowdin_link, 'id2': get_crowdin_link})

In [ ]:
df[df['translated1'] > 0].style.format({'id1': get_crowdin_link, 'id2': get_crowdin_link})